# module import

In [ ]:
# src 폴더의 kskwClass.py에 정의된 모듈 호출
from src.kskwClass import support            # 연관상품추천 module
from src.kskwClass import collab_filtering   # 잠재요인 협업필터링 추천 module
from src.kskwClass import ESG_cal            # ESG score 계산 module

# basic libraies
import pandas as pd
import numpy as np
import gc
import os
import warnings
warnings.filterwarnings("ignore")

import joblib

# for ESG_cal module
import time
from datetime import datetime
import random
random.seed(1004)

# for support module
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder



data_path = "../3. 경서경욱_데이터 및 모델 세이브 파일/dataset/"
model_path = "../3. 경서경욱_데이터 및 모델 세이브 파일/model/"

# Modul use

## 1. 잠재요인 협업 필터링을 사용한 상품 추천 - collab_filtering

In [ ]:
# Read data & model

model = joblib.load(model_path+'svd.pkl')  # 저장된 SVD 모델
data = pd.read_csv(data_path + "custom_dataset/collab_filtering_valset.csv", index_col = 0)  # cust-item-rating dataset
product = pd.read_csv(data_path + "origin_dataset/LPOINT_BIG_COMP_04_PD_CLAC.csv")  # 전체 상품 dataset

pro_list = list(product.pd_nm.unique()) # 전체 상품 리스트
val_cust_list = data.custID.unique().tolist() # 상품 추천을 받을 고객 명단 (시범용)

In [ ]:
# 검증용 고객들에 대한 추천 상품 추출

for custID in val_cust_list:
    filter_recommen = collab_filtering(model, data, pro_list, custID)
    print(custID, "님의 추천 상품 리스트입니다.")
    filter_recommen.recomm_item_by_surprise()
    print("\n")

## 2. 연관규칙분석 -> 연관 상품을 추천하는 경우

In [ ]:
# Read data & model

pdde = pd.read_csv(data_path + "origin_dataset/LPOINT_BIG_COMP_02_PDDE.csv") # 유통사 상품 구매 내역

# 상품코드에 따른 상품명 labeling feature
pdde = pdde.merge(product, on = "pd_c", how = "left")
pdde = pdde.drop("pd_c", axis = 1)

#### 장바구니, 찜한 상품, 최근 결제 내역에 '남성의류'가 있다면?

In [ ]:
product = "남성티셔츠"
recommend_class = support(pdde)
recommend_class.pro_recommend(product)

## 3. ESG score 계산

In [ ]:
# Read data & model

ESG = pd.read_csv(data_path + "custom_dataset/ESG_score.csv", index_col=0)

In [ ]:
esg = ESG_cal(ESG, "M816318679")

* 누적된 ESG score 확인

In [ ]:
esg.lookup_score()

* 새로운 결제 내역에 대한 ESG score 확인 + 데이터셋에 추가

In [ ]:
esg.new_pay()

In [ ]:
esg.score_data

## 4. 시계열 예측 (다음 결제일 예측)

In [ ]:
catmodel = joblib.load(model_path + "catboost_Lpay.pkl")
test_set = pd.read_csv(data_path + "custom_dataset/feature_test.csv")
test = test_set.drop(["cust", "date"], axis = 1)

test.head(5)

In [ ]:
test_set["predict"] = catmodel.predict(test)

In [ ]:
test_set[["cust", "predict"]]

* 예측값(predict열)은 시간단위이다.